In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

### Perlin Noise
1. Gride Definition
2. Dot product
3. Interpolation

### Grid definition

In [2]:
IMAGE_WIDTH = 500
IMAGE_HEIGHT = 500

In [3]:
# pixels per grid
grid_width = 5
grid_height = 5
grid_size = (grid_height, grid_width)

In [4]:
print(IMAGE_HEIGHT / grid_height, 'grids in a row')
print(IMAGE_WIDTH / grid_width, 'grids in a col')

grids_per_row = int(IMAGE_HEIGHT / grid_height)
grids_per_col = int(IMAGE_WIDTH / grid_width)

100.0 grids in a row
100.0 grids in a col


### Dot product

In [5]:
# assert(IMAGE_HEIGHT / grid_height == IMAGE_WIDTH / grid_width)
print('total number of vectors:', (IMAGE_HEIGHT / grid_height + 1) * (IMAGE_WIDTH / grid_width + 1))

vec_shape = (int(IMAGE_WIDTH / grid_width + 1), int(IMAGE_WIDTH / grid_width + 1))

# thetas = np.random.uniform(size=vec_shape) * 360 
thetas = np.random.choice([45, 135, 225, 315], size=vec_shape)

total number of vectors: 10201.0


In [6]:
def pol2cart(phi, rho=1):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return np.array([x, y])

In [7]:
vecs = np.array([pol2cart(t) for t in np.nditer(thetas)])
vecs = vecs.reshape((*vec_shape, 2))
# vecs is a 2d matrix where each entry represents the vector at that point
print(vecs.shape)

(101, 101, 2)


In [8]:
# helper function that returns the 4 position vectors 
# row, col are index values
# grid size is a 2d tuple of dims of grid
def get_pos_vec(row, col, grid_size):
    x = np.linspace(0, 1, grid_size[0])[row]
    y = np.linspace(0, 1, grid_size[1])[col]
    
    # 4 pos vectors, 
    # order: top left, top right, bottom left, bottom right
    a = (x, y)
    b = (x-1, y)
    c = (x, y-1)
    d = (x-1, y)
    
    return np.array([a, b, c, d])

In [9]:
get_pos_vec(1, 2, (4,5))

array([[ 0.33333333,  0.5       ],
       [-0.66666667,  0.5       ],
       [ 0.33333333, -0.5       ],
       [-0.66666667,  0.5       ]])

In [10]:
# helper function that returns the 4 gradient vector of a given grid
# row and col are index values
# vecs is a 2d matrix where each entry represents the vector at that point (height, width)
def get_grad_vec(row, col, vecs, vec_shape, grid_size):
    
    row = int(row / grid_size[0])
    col = int(col / grid_size[1])
    
#     print(row, col)
    
    assert(row < vec_shape[0] and col < vec_shape[1])
    
    # 4 grad vectors, 
    # order: top left, top right, bottom left, bottom right
    a = vecs[row][col]
    b = vecs[row][col + 1]
    c = vecs[row + 1][col]
    d = vecs[row + 1][col + 1]
    
    return np.array([a, b, c, d])

In [11]:
get_grad_vec(0, 0, vecs, vec_shape, (grid_height, grid_width))

array([[ 0.36731937, -0.93009488],
       [ 0.6669156 ,  0.74513326],
       [-0.99608784,  0.08836869],
       [-0.99608784,  0.08836869]])

In [12]:
temp1 = get_pos_vec(1, 2, (4,5))
temp2 = get_grad_vec(0, 0, vecs, vec_shape, grid_size)

for i, j in zip(temp1, temp2):
    print(np.dot(i, j))

-0.34260764975884767
-0.07204376798452183
-0.3762136214323957
0.7082428998127908


### interpolation

In [13]:
# helper function that interpolates the 4 values of a dot product
def lin_interp(fracX, fracY, dot_prods):
    a, b, c, d = tuple(dot_prods)
    ab = a + fracX * (b-a)
    cd = c + fracY * (d-c)
    val = ab + fracY * (cd - ab)
    
    return val

In [14]:
pixel_vals = []
for row in range(IMAGE_HEIGHT):
    for col in range(IMAGE_WIDTH):
        grad_vec = get_grad_vec(row, col, vecs, vec_shape, (grid_height, grid_width))
        
#         print('***', row, col, '***')
        
        for i in range(grid_height):
            for j in range(grid_width):
                pos_vec = get_pos_vec(i, j, (grid_height, grid_width))
                fracX, fracY = pos_vec[0]
                
                px = []
                
                for grad, pos in zip(grad_vec, pos_vec):
#                     print(np.dot(grad, pos))
                    px.append(np.dot(grad, pos))
                val = lin_interp(fracX, fracY, px)
                
        pixel_vals.append(val)
pixel_vals = np.array(pixel_vals).reshape(IMAGE_HEIGHT, IMAGE_WIDTH)

KeyboardInterrupt: 

In [ ]:
pixel_vals.shape

In [ ]:
plt.imshow(pixel_vals, cmap='Blues')

In [ ]:
plt.imshow(np.random.uniform(size=(int(IMAGE_HEIGHT/grid_height), int(IMAGE_WIDTH/grid_width))), cmap='Blues')